In [1]:
import os
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(300)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [2]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    (pl.col('datum').dt.year() >= 1918)
    & 
    (~pl.col('komora_komplet').str.contains('(?i)senát')) 
    & 
    (~pl.col('komora_komplet').str.contains('(?i)společn'))
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).with_columns(
    pl.when(pl.col("soubor") == "1996ps_stenprot_006schuz_s006024.htm").then(pl.lit(2006)).otherwise(pl.col('obdobi')).alias('obdobi')
).collect()

NameError: name 'pl' is not defined

In [20]:
df.select(pl.col("komora")).unique()

komora
str
"""ns"""
"""cnr"""
"""fs"""
"""psse"""
"""ps"""
"""pns"""


In [21]:
df.select(pl.col("komora_komplet")).unique()

komora_komplet
str
"""PČR, PS 2021-..."""
"""Národní shromáždění československé 1918 – 1920"""
"""Česká národní rada 1992 – 1992"""
"""FS ČSSR 1971-1976"""
"""ČNR 1971-1976"""
"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013"""
"""PČR, PS 2006-2010"""
"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006"""
"""NS RČS 1929-1935, Poslanecká sněmovna"""


In [29]:
slov_po_roce = df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('pocet_slov').sum())

In [121]:
alt.Chart(
    slov_po_roce.to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [25]:
def cetnost_po_roce(popisek, hledani):
    return df.filter(
        pl.col('text').str.contains(hledani)
    ).with_columns(
        pl.col('text').str.count_matches(hledani).alias('pocet_vyskytu')
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col('pocet_vyskytu').sum()
    ).join(
        slov_po_roce, on='datum', how='right'
    ).with_columns(
        (pl.col('pocet_vyskytu') / pl.col('pocet_slov')).alias('podil')
    ).select(
        pl.col(['datum','podil'])
    ).with_columns(
        pl.lit(popisek).alias('co')
    )

In [31]:
cetnost_po_roce("hasič","(?i)hasič").to_pandas()

,datum,podil,co
0,1918-01-01,0.000005,hasič
1,1919-01-01,0.000002,hasič
2,1920-01-01,0.000001,hasič
3,1921-01-01,0.000004,hasič
4,1922-01-01,0.000002,hasič
...,...,...,...
96,2021-01-01,0.000062,hasič
97,2022-01-01,0.000171,hasič
98,2023-01-01,0.000064,hasič
99,2024-01-01,0.000070,hasič


In [35]:
alt.Chart(
    cetnost_po_roce("hasič","(?i)hasič").to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [49]:
alt.Chart(
    cetnost_po_roce("sekretářka","(?i)sekretářk").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [47]:
alt.Chart(
    cetnost_po_roce("prostitutk","(?i)prostitutk").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [45]:
alt.Chart(
    cetnost_po_roce("záchranář","(?i)záchranář").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [43]:
alt.Chart(
    cetnost_po_roce("záchranář","(?i)záchranář").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [51]:
alt.Chart(
    cetnost_po_roce("manažer","(?i)manažer").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [53]:
alt.Chart(
    cetnost_po_roce("farář","(?i)farář").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [57]:
alt.Chart(
    cetnost_po_roce("učitel","(?i)učitel").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [59]:
alt.Chart(
    cetnost_po_roce("překladatel","(?i)překladatel").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [61]:
alt.Chart(
    cetnost_po_roce("voják","(?i)vojá[kc]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [63]:
alt.Chart(
    cetnost_po_roce("programátor","(?i)programáto[rř]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [65]:
alt.Chart(
    cetnost_po_roce("novinář","(?i)novinář").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [69]:
alt.Chart(
    cetnost_po_roce("pilot","(?i)pilot[^n]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [73]:
alt.Chart(
    cetnost_po_roce("právník","(?i)právn[ií][ck]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [75]:
alt.Chart(
    cetnost_po_roce("advokát","(?i)advokát").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [77]:
alt.Chart(
    cetnost_po_roce("dělník","(?i)děln[ií][ck]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [79]:
alt.Chart(
    cetnost_po_roce("zedník","(?i)zedn[ií][ck]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [81]:
alt.Chart(
    cetnost_po_roce("soustružník","(?i)soustružn[ií][ck]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [83]:
alt.Chart(
    cetnost_po_roce("průvodčí","(?i)průvodčí").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [85]:
alt.Chart(
    cetnost_po_roce("policista","(?i)policist").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [87]:
alt.Chart(
    cetnost_po_roce("letuška","(?i)letušk").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [91]:
alt.Chart(
    cetnost_po_roce("úředník","(?i)úředn[ií][kc]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [93]:
alt.Chart(
    cetnost_po_roce("kosmonaut","(?i)kosmonaut").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [95]:
alt.Chart(
    cetnost_po_roce("astronaut","(?i)astronaut").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [105]:
alt.Chart(
    cetnost_po_roce("hokejista","(?i)hokejist").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [107]:
alt.Chart(
    cetnost_po_roce("farmář","(?i)farmář[^s]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [109]:
alt.Chart(
    cetnost_po_roce("traktorista","(?i)traktorist").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [111]:
alt.Chart(
    cetnost_po_roce("horník","(?i)horní[ck]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [113]:
alt.Chart(
    cetnost_po_roce("havíř","(?i)havíř[o]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [123]:
alt.Chart(
    cetnost_po_roce("lékař","(?i)lékař").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [125]:
alt.Chart(
    cetnost_po_roce("zubař","(?i)zubař").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [127]:
alt.Chart(
    cetnost_po_roce("silničář","(?i)silničář").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [129]:
alt.Chart(
    cetnost_po_roce("vychovatel","(?i)vychovatel").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [131]:
alt.Chart(
    cetnost_po_roce("pediatr","(?i)pediatr").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [135]:
alt.Chart(
    cetnost_po_roce("řemeslník","(?i)řemeslní[kc]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [137]:
alt.Chart(
    cetnost_po_roce("kadeřník","(?i)kadeřn[íi][kc]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [139]:
alt.Chart(
    cetnost_po_roce("číšník","(?i)číšn[ií][kc]").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [141]:
alt.Chart(
    cetnost_po_roce("podnikatel","(?i)podnikatel").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [97]:
def hledej(text, n=3, delka=200):
    nalezy = df.sort(by="datum").filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-delka:] if len(before) > delka else before
        after_snippet = after[:delka] if len(after) > delka else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [101]:
import re

In [103]:
hledej("[Aa]stronaut")

Kdo: Poslanec Vojtěch Filip
Kde: 1996ps_psse_stenprot_001schuz_s001003.htm
Kdy: 1936-11-30
Co: 1992 a 1993 pracoval rok v Polárním geofyzikálním ústavu Ruské akademie věd v Apatitech a v roce 1995 tři měsíce na Athénské univerzitě. Pan ing. Stanislav Fischer, CSc., je řádným členem Mezinárodní astronautické akademie v Paříži, Mezinárodní astronomické unie, Evropské astronomické společnosti v Ženevě, doživotním členem Americké geofyzikální unie. Působí také v New York Academy of Science a v dalších v
***
Kdo: Posl. Vecker
Kde: 1954ns_stenprot_036schuz_s036005.htm
Kdy: 1960-02-17
Co:  proto rozhořčeně odsuzuje ty, kteří proti vůli národů chtějí zůstávat na pozicích studené války. Mladí lidé dychtivě sledují veliké objevy vědy a techniky, jsou vzrušeni představou o cestách prvních astronautů. A taková je mládež v celém světě. Přesto, že v kapitalistickém Západě vychovávají mládež k pesimismu a beznaději, zabijáctví a cynismu, poznali jsme mnoho mladých lidí, jimž imponuje nikoliv výbuch
*